In [1]:
import numpy as np
import sys
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_PFCI import Determinant
from helper_cqed_rhf import cqed_rhf
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)

In [7]:
# options for mgf
mol_str = """
Mg
H 1 2.2
symmetry c1
1 1
"""

options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 4.75 / psi4.constants.Hartree_energy_in_eV,
    'lambda_vector' : np.array([0, 0, 0.0125]),
    'ci_level' : 'cis',
    'ignore_coupling' : False,
    'number_of_photons' : 1,
    'natural_orbitals' : False,
    'nact_orbs' : 0,
    'nact_els' : 0
}

mol = psi4.geometry(mol_str)

psi4.set_options(options_dict)

#energy from psi4numpy
expected_mghp_eg = -199.8639591041915
expected_mghp_e1 = -199.6901102832973

test_pf = PFHamiltonianGenerator(
    mol_str,
    options_dict,
    cavity_dict
)

e_fci, wavefunctions = np.linalg.eigh(test_pf.H_PF)

print(e_fci[1]-e_fci[0])

#assert psi4.compare_values(e_fci[0], expected_mghp_eg)
#assert psi4.compare_values(e_fci[1], expected_mghp_e1)


Start SCF iterations:

Canonical RHF One-electron energy = -281.6658725092593158
CQED-RHF One-electron energy      = -281.6641286091231109
Nuclear repulsion energy          = 2.8864211491090908
Dipole energy                     = 0.0009465637016117
SCF Iteration   1: Energy = -199.8633303817346700   dE = -1.99863E+02   dRMS = 2.52850E-05
SCF Iteration   2: Energy = -199.8633315502319476   dE = -1.16850E-06   dRMS = 9.55045E-06
SCF Iteration   3: Energy = -199.8633316895825374   dE = -1.39351E-07   dRMS = 3.61517E-06
SCF Iteration   4: Energy = -199.8633317095168138   dE = -1.99343E-08   dRMS = 1.37921E-06
SCF Iteration   5: Energy = -199.8633317124263726   dE = -2.90956E-09   dRMS = 5.28065E-07
SCF Iteration   6: Energy = -199.8633317128548299   dE = -4.28457E-10   dRMS = 2.02751E-07
SCF Iteration   7: Energy = -199.8633317129183808   dE = -6.35509E-11   dRMS = 7.79875E-08
SCF Iteration   8: Energy = -199.8633317129279305   dE = -9.54969E-12   dRMS = 3.00308E-08
SCF Iteration   9: Ene

In [10]:
qed_rhf_e = -199.8633317129295790
gs_e = qed_rhf_e + -0.000250831265
LP_e = qed_rhf_e + 0.165570838034
UP_e = qed_rhf_e + 0.182666685009

print(gs_e)
print(LP_e)
print(UP_e)

print(e_fci[0]-qed_rhf_e)
print(e_fci[1]-qed_rhf_e)
print(e_fci[2]-qed_rhf_e)
print(e_fci[3]-qed_rhf_e)
print(e_fci[4]-qed_rhf_e)
print(e_fci[5]-qed_rhf_e)


-199.86358254419457
-199.69776087489558
-199.68066502792058
-0.0002508312595637108
0.02445474970161854
0.16557083778926085
0.18261984513992502
0.18261984514001028
0.18266668449453505


In [ ]:
# photon energy
#omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
omega_val = 0

print(omega_val)

# lambda vector
lambda_vector = np.array([0., 0., 0.0])
#lambda_vector = np.array([0.5, 0.5, 0.5])

n_act_orb=5
#number of active electrons
n_act_el=4
# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(mol_str, options_dict, cavity_options)


In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

# davidson eigenvalues and eigenvectors are stored in attributes
# H2_PF.cis_e and H2_PF.cis_c for now!


In [ ]:
print("checking to see if the Davidson Roots Agree!")
print(e_fci[:14])
print(H2_PF.cis_e[:14] )
print(np.allclose(e_fci[:14] , H2_PF.cis_e[:14]))


In [ ]:

H2_PF.calc1RDMfromCIS(wavefunctions[:,14])
print(np.trace(H2_PF.Dij))
print(np.trace(H2_PF.Dab))
print(np.trace(H2_PF.D1))



e1_test_rdm = np.einsum("pq,pq->", H2_PF.Hspin, H2_PF.D1)

temp=np.einsum("pq,q->p", H2_PF.H_1E,wavefunctions[:,14])
e1_test_wfn = np.dot(wavefunctions[:,14].T,temp)




In [ ]:
print(e1_test_rdm)
print(e1_test_wfn-H2_PF.Enuc)
print(e1_test_rdm-e1_test_wfn+H2_PF.Enuc)